In [1]:
import base64
import datetime
import gc
import json
import pickle
import random
import time

import numpy as np
import pandas as pd
import sklearn
import sklearn.decomposition
import xgboost
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.5)
rcParams['figure.figsize'] = 12, 8
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [48]:
d = pd.read_csv('tmp/dmy_ho_train_ind_recibo_ult1.csv', nrows=200000)
l = d.target.copy()
del d['customer_id']
del d['target']
del d['target_name']

lim_train = len(d)/2
tr = d[:lim_train].copy()
trl = l[:lim_train].copy()
te = d[lim_train:].copy()
tel = l[lim_train:].copy()

m = XGBClassifier(
        n_estimators=100,
        learning_rate=0.01,
        max_depth=9,
        gamma=0,
        min_child_weight=1,
        scale_pos_weight=1.0,
        nthread=3)
m.fit(tr, trl)

p = m.predict(te)

print 'test labels:'
print tel.value_counts()

print 'ROC AUC score:'
print roc_auc_score(tel, p)

cmx = pd.DataFrame(confusion_matrix(tel, p))
cmx.index.name='actual'
cmx.columns.name='predicted'
print 'confusion matrix:'
print cmx

test labels:
0.0    83612
1.0    16388
Name: target, dtype: int64
ROC AUC score:
0.802633827969
confusion matrix:
predicted      0      1
actual                 
0          80730   2882
1           5904  10484


In [12]:
components = 5

d = pd.read_csv('tmp/dmy_ho_train_ind_recibo_ult1.csv', nrows=200000)
cids = d.customer_id.copy()
l = d.target.copy()
target_names = d.target_name.copy()
del d['customer_id']
del d['target']
del d['target_name']

for c in d.columns.tolist():
    if c.endswith('_no'):
        del d[c]

d.fillna(0, inplace=True)
d = sklearn.preprocessing.normalize(d)
pca = sklearn.decomposition.PCA(n_components=components)
d = pca.fit_transform(d)
pca.explained_variance_ratio_

d = pd.DataFrame(d)
d.columns = ['pc_{0}'.format(i) for i in range(components)]

In [51]:
lim_train = len(d)/2
tr = d[:lim_train].copy()
trl = l[:lim_train].copy()
te = d[lim_train:].copy()
tel = l[lim_train:].copy()

m = sklearn.neighbors.KNeighborsClassifier(n_neighbors=3, n_jobs=3)
m.fit(tr, trl)

p = m.predict(te)

print 'test labels:'
print tel.value_counts()

print 'ROC AUC score:'
print roc_auc_score(tel, p)

cmx = pd.DataFrame(confusion_matrix(tel, p))
cmx.index.name='actual'
cmx.columns.name='predicted'
print 'confusion matrix:'
print cmx

test labels:
0.0    83612
1.0    16388
Name: target, dtype: int64
ROC AUC score:
0.58911254025
confusion matrix:
predicted      0     1
actual                
0          77723  5889
1          12313  4075


In [14]:
lim_train = len(d)/2
tr = d[:lim_train].copy()
trl = l[:lim_train].copy()
te = d[lim_train:].copy()
tel = l[lim_train:].copy()

m = sklearn.svm.SVC(kernel='linear', class_weight='balanced')
m.fit(tr, trl)

p = m.predict(te)

print 'test labels:'
print tel.value_counts()

print 'ROC AUC score:'
print sklearn.metrics.roc_auc_score(tel, p)

cmx = pd.DataFrame(sklearn.metrics.confusion_matrix(tel, p))
cmx.index.name='actual'
cmx.columns.name='predicted'
print 'confusion matrix:'
print cmx

test labels:
0.0    83612
1.0    16388
Name: target, dtype: int64
ROC AUC score:
0.544803052889
confusion matrix:
predicted      0     1
actual                
0          78145  5467
1          13848  2540
